In [1]:
import tbapy
import numpy as np
import pandas as pd
tba = tbapy.TBA('70PhghWVToSpFoaK0scpRo6PCL7iKUff8tPhQXj4LrhQgwA60t7SBf5Tt4XvefzX')
year = 2019
match_input_file = "match_data.csv"
output_file = "team_data.csv"

In [2]:
# first, let's get a list of teams to query
all_teams = set([])

match_in = open(match_input_file, 'r')
for line in match_in.readlines()[1:]:
    all_teams.add(line.split(",")[2])
    
all_teams = sorted(all_teams, key=lambda t : int(t[3:]))
print(all_teams[:10])

['frc1', 'frc4', 'frc8', 'frc11', 'frc16', 'frc20', 'frc21', 'frc23', 'frc25', 'frc27']


In [3]:
# Setting up storage datastructure
team_data = {}
for team in all_teams:
    team_data[team] = {}

In [4]:
# PULLING ELO
elo_data = pd.read_csv('elo_data.csv')
def getTeamElo(team):
    team_num = int(team[3:])
    row = elo_data.loc[elo_data['Team'] == team_num]
    return row.values[0][1]
for team in all_teams:
    team_data[team]['elo'] = getTeamElo(team)

In [14]:
# Adding # event wins & seasons
for team in all_teams:
    team_data[team]['event_wins'] = 0
    team_data[team]['1_wins'] = 0 # win as alliance captain
    team_data[team]['2_wins'] = 0 # win as first pick
    team_data[team]['3_wins'] = 0 # win as second pick
for yr in list(range(2005, 2019)): # wins from 2005-2018
    all_events = list(filter(lambda e : e.event_type >= 0 and e.event_type <= 5, tba.events(yr))) # all regular season events
    for event_key in [e.key for e in all_events]:
        win_awards = list(filter(lambda a : a.award_type == 1,tba.event_awards(event_key)))
        win_award = win_awards[0] if len(win_awards) == 1 else None
        if win_award == None:
            print(f"No event winner found for {event_key}")
            continue
        for i, win_dict in enumerate(win_award.recipient_list):
            team_key = win_dict['team_key']
            if team_key not in all_teams:
                continue
            team_data[team_key]['event_wins'] += 1
            if i + 1 <= 3:
                team_data[team_key][f"{i + 1}_wins"] += 1

No event winner found for 2006wa


In [17]:
# Adding # seasons participated
for team in all_teams:
    team_data[team]['num_seasons'] = len(tba.team_years(team))

In [19]:
# Adding event wins per season
for team in all_teams:
    team_data[team]["event_wins_per_season"] = team_data[team]["event_wins"]/team_data[team]['num_seasons']

In [20]:
print(team_data['frc4944'])

{'elo': 1667, 'event_wins': 0, '1_wins': 0, '2_wins': 0, '3_wins': 0, 'num_seasons': 7, 'event_wins_per_season': 0.0}


In [23]:
writer = open(output_file, 'w')
cols = list(team_data['frc1'].keys())
writer.write(",".join(["team"] + cols) + '\n')
for team in all_teams:
    writer.write(",".join([team] + [str(team_data[team][c]) for c in cols]) + '\n')
writer.close()